In [ ]:
import os
import numpy as np
from typing import Optional
from scipy.linalg import expm, block_diag
import matplotlib.pyplot as plt
from im import coupled_dynamics
from constants import sigmax, sigmay
from cli_utils import _hdf2im, _hdf2trained_im

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 20,
})
script_dir = os.path.dirname(os.path.abspath(''))

# ----------------------------------------------------------------------------------- #

def _exact_charge_dynamics(
    initial_coupling: np.array,
    final_coupling: np.array,
    env_coupling: np.array,
    total_time_steps: int,
    control_point: Optional[int],
    env_length: int,
) -> np.array:
    state = np.diag(np.array([1] * env_length + [0] * (env_length + 4)))
    env_gate = expm(1j * env_coupling * 2 * sigmax)
    initial_gate = expm(1j * initial_coupling * 2 * sigmax)
    final_gate = expm(1j * final_coupling * 2 * sigmax)
    u_e_initial = block_diag(*([env_gate] * (env_length // 2) + [initial_gate] + [env_gate] * (env_length // 2) + [np.eye(2)]))
    u_e_final = block_diag(*([env_gate] * (env_length // 2) + [final_gate] + [env_gate] * (env_length // 2) + [np.eye(2)]))
    u_o = block_diag(*([np.eye(1)] + [env_gate] * env_length + [np.eye(3)]))
    kicking = block_diag(*([env_gate] * (env_length // 2) + [np.zeros((2, 2))] + [env_gate] * (env_length // 2) + [np.zeros((2, 2))]))
    kicking[2 * env_length + 2, env_length] = 1
    kicking[2 * env_length + 3, env_length + 1] = 1
    kicking[env_length, 2 * env_length + 2] = 1
    kicking[env_length + 1, 2 * env_length + 3] = 1
    result = [state[env_length + 1, env_length + 1]]
    if control_point is None:
        for _ in range(total_time_steps):
            state = u_o @ state @ u_o.T.conj()
            state = u_e_initial @ state @ u_e_initial.T.conj()
            result.append(state[env_length + 1, env_length + 1])
    else:
        for _ in range(control_point):
            state = u_o @ state @ u_o.T.conj()
            state = u_e_initial @ state @ u_e_initial.T.conj()
            result.append(state[env_length + 1, env_length + 1])
        state = u_o @ state @ u_o.T.conj()
        state = kicking @ state @ kicking.T.conj()
        result.append(state[env_length + 1, env_length + 1])
        for _ in range(total_time_steps - control_point - 1):
            state = u_o @ state @ u_o.T.conj()
            state = u_e_final @ state @ u_e_final.T.conj()
            result.append(state[env_length + 1, env_length + 1])
    return np.array(result)

# ----------------------------------------------------------------------------------- #

plotting_config = {
    "left_im": "J_0.1_do_xy/2023-10-04_11:33:29+0200",  # left im's stump
    "right_im": "J_0.1_up_xy/2023-10-04_11:33:54+0200",  # right im's stumps
    "initial_J": 0.1,  # initial coupling between impurity's parts
    "final_J": 0.0,  # final coupling between impurity's parts
    "control_time_moment": 20,  # a discrete time step when control sequence starts
    "dens":  # initial impurity's density matrix
        np.array([
            0., 0., 0., 0.,
            0., 0., 0., 0.,
            0., 0., 0., 0.,
            0., 0., 0., 1., 
        ]),
}
output_path = f"{script_dir}/../6467a173cb828c16e8ae9ac3/xx_memory_probing.pdf"

# ----------------------------------------------------------------------------------- #

left_im_path = f"{script_dir}/experiments/output/physical_im/{plotting_config['left_im']}"
right_im_path = f"{script_dir}/experiments/output/physical_im/{plotting_config['right_im']}"

h = plotting_config["initial_J"] * np.kron(sigmax, sigmax) +\
    plotting_config["initial_J"] * np.kron(sigmay, sigmay)
u = expm(-1j * h).reshape((2, 2, 2, 2))
initial_phi = np.tensordot(u, u.conj(), axes=0)
initial_phi = initial_phi.transpose((0, 4, 1, 5, 2, 6, 3, 7)).reshape((4, 4, 4, 4))

h = plotting_config["final_J"] * np.kron(sigmax, sigmax) +\
    plotting_config["final_J"] * np.kron(sigmay, sigmay)
u = expm(-1j * h).reshape((2, 2, 2, 2))
final_phi = np.tensordot(u, u.conj(), axes=0)
final_phi = final_phi.transpose((0, 4, 1, 5, 2, 6, 3, 7)).reshape((4, 4, 4, 4))

tr = np.array([1, 0, 0, 1])
tr = np.tensordot(tr, tr, axes=0).reshape((16,))
zero_fermions = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
knocking = np.tensordot(zero_fermions, tr, axes=0).reshape((4, 4, 4, 4))

im_exact1 = _hdf2im(left_im_path)
im_exact2 = _hdf2im(right_im_path)
im_trained1 = list(reversed(_hdf2trained_im(left_im_path)))
im_trained2 = list(reversed(_hdf2trained_im(right_im_path)))

control_sequence = plotting_config["control_time_moment"] * [initial_phi] +\
    [knocking] + (len(im_exact1) - plotting_config["control_time_moment"] - 1) * [final_phi]
rhos_gen_controlled = np.array(coupled_dynamics(im_exact1, im_exact2, control_sequence, plotting_config["dens"]))
rhos_trained_controlled = np.array(coupled_dynamics(im_trained1, im_trained2, control_sequence, plotting_config["dens"]))
rhos_gen = np.array(coupled_dynamics(im_exact1, im_exact2, len(im_exact1) * [initial_phi], plotting_config["dens"]))
rhos_trained = np.array(coupled_dynamics(im_trained1, im_trained2, len(im_exact1) * [initial_phi], plotting_config["dens"]))
controlled_gen_charge = rhos_gen_controlled[:, 0, 0] + rhos_gen_controlled[:, 1, 1]
controlled_trained_charge = rhos_trained_controlled[:, 0, 0] + rhos_trained_controlled[:, 1, 1]
exact_charge = _exact_charge_dynamics(
    plotting_config["initial_J"],
    0.,
    plotting_config["initial_J"],
    len(im_exact1),
    None,
    240,
)
controlled_exact_charge = _exact_charge_dynamics(
    plotting_config["initial_J"],
    plotting_config["final_J"],
    plotting_config["initial_J"],
    len(im_exact1),
    plotting_config["control_time_moment"],
    240,
)
gen_charge = rhos_gen[:, 0, 0] + rhos_gen[:, 1, 1]
trained_charge = rhos_trained[:, 0, 0] + rhos_trained[:, 1, 1]
fig = plt.figure(figsize=(2 * 6.4 / 1.5, 4.8 / 1.5))
plt.subplot(1, 2, 1)
plt.plot(gen_charge, 'r', label=r"${\rm ab \ initio}$")
plt.plot(exact_charge, 'k--', label=r"${\rm exact}$")
plt.plot(trained_charge , 'b--', label=r"$N=7.5\cdot 10^6$")
plt.legend(frameon=False, labelspacing=0.02, handlelength=1.5, fontsize="15")
plt.subplot(1, 2, 2)
plt.plot(controlled_gen_charge, 'r')
plt.plot(controlled_exact_charge, 'k--')
plt.plot(controlled_trained_charge , 'b--')
plt.legend(frameon=False, labelspacing=0.02, handlelength=1.5, fontsize="15")
plt.axvline(x = plotting_config["control_time_moment"], color='k', linewidth=0.75)
fig.subplots_adjust(hspace=0.3)
fig.text(0.5, -0.1, r"${\rm Time}$", ha='center')
fig.text(0.04, 0.5, r"$n$", va='center', rotation='vertical')
fig.show()

In [ ]:
fig.savefig(output_path, bbox_inches = 'tight')